<a href="https://colab.research.google.com/github/antoinexp/markov-chains-COM-516/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is provided as a starting point to help you generate random instances G1 and G2 as mentioned in the handout.

You are free to use and modify it at your own convenience.

---



In [1]:
import scipy.stats as st
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

In [2]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

In [3]:
%matplotlib widget

plt.rcParams['figure.figsize'] = [8.0, 6.0]
plt.rcParams['figure.dpi'] = 80
plt.rcParams['savefig.dpi'] = 100

plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 'large'
plt.rcParams['figure.titlesize'] = 'medium'

In [4]:
class DatasetGenerator(ABC):
    def __init__(self, N=100):
        self.N = N
        self.x = None
        self.v = None
        self.refresh()
    
    @abstractmethod
    def refresh(self):
        pass

In [5]:
class G1(DatasetGenerator):
    def refresh(self):
        self.x = np.random.rand(self.N, 2)
        self.v = np.random.rand(self.N)

In [6]:
class G2(DatasetGenerator):
    def refresh(self):
        self.x = np.random.rand(self.N, 2)
        self.v = np.exp(np.random.randn(self.N) * 1.3 - 0.85)

### Uniform distribution ($\mathcal U([0,1])$)

In [7]:
g1 = G1()

Examples:

In [8]:
# Plot a histogram of the v array
plt.figure()
plt.hist(g1.v, bins=30)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# plot the position of the points
plt.figure()
plt.scatter(g1.x[:,0], g1.x[:,1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

You can refresh the dataset

In [10]:
g1.refresh() # generate a new dataset

In [11]:
plt.figure()
plt.hist(g1.v, bins=30)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Or for instance, you can generate 10 instances and compute the average position of all the points:

In [12]:
m = np.array([0., 0.])

for _ in range(10):
    g1.refresh() # refresh the dataset
    m += 0.1*g1.x.mean()

print(m)

[0.49569313 0.49569313]


### Test on log-normal distribution

In [13]:
g2 = G2()

Example:

you can use g2 to generate an instance of the lognormal distribution

In [14]:
plt.figure()
plt.hist(g2.v, bins=30)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
g2.refresh() # to generate a new x and v

In [16]:
plt.figure()
plt.hist(g2.v, bins=30)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---

### Metropolis Hastings

In [17]:
def objective(v, x, lam, indicator):
    n = len(indicator)
    assert len(v) == n
    assert x.shape == (n, 2)
    v_sum = np.sum(v * indicator)
    cities = x[indicator, :]
    return 0